In [48]:
import pandas as pd

In [49]:
data = pd.read_excel("rawdata.xlsx")

In [50]:
data.sample(5)

,date,time,sensor,location,number,activity,position,location.1
56,2024-01-16,01:00:20,1.0,A1,1,placed,inside,A1
17,2024-01-16,11:48:20,1.0,A18,1,picked,inside,A18
166,2024-01-18,12:05:22,0.0,A4,2,placed,inside,A4
106,2024-01-18,11:50:20,1.0,A23,1,placed,inside,A23
111,2024-01-18,12:35:30,1.0,A28,1,placed,inside,A28


Task: Use rawdata as input and derive
1. Datewise total duration for each inside and outside.
2. Datewise number of picking and placing activity done.
Sample output shared in output sheet.


In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        175 non-null    datetime64[ns]
 1   time        175 non-null    object        
 2   sensor      174 non-null    float64       
 3   location    175 non-null    object        
 4   number      175 non-null    int64         
 5   activity    175 non-null    object        
 6   position    175 non-null    object        
 7   location.1  175 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 11.1+ KB


In [52]:
# Replace 'Inside' with 'inside'
data['position'] = data['position'].replace('Inside', 'inside')

In [53]:
#Converting the time column in total seconds
data["time"] = data["time"].astype(str)

def time_to_hours(time_str):
    # Split the time string into hours, minutes, and seconds
    hours, minutes, seconds = map(int, time_str.split(':'))
    
    # Convert each part to seconds
    total_seconds = hours * 3600 + minutes * 60 + seconds
    total_hours = round(total_seconds / 3600, 1)
    
    return total_hours


data["time_in_hours"] = data["time"].apply(time_to_hours)

In [54]:
# Group the data by date and position, then sum the time_in_hours for each group
total_time_by_date_position = data.groupby(['date', 'position'])['time_in_hours'].sum()

In [55]:
# Group the data by date and activity, then count the occurrences of each activity
activity_count_by_date = data.groupby(['date', 'activity'])['activity'].count()

In [70]:
total_time_df = total_time_by_date_position.unstack(fill_value=0)
activity_count_df = activity_count_by_date.unstack(fill_value=0)

merged_df = total_time_df.merge(activity_count_df, left_index=True, right_index=True)

# Rename the columns
merged_df = merged_df.rename(columns={'inside': 'total_duration_inside', 'outside': 'total_duration_outside'})

merged_df

,total_duration_inside,total_duration_outside,picked,placed
date,,,,
2024-01-16,243.4,0.0,40,40
2024-01-17,216.8,0.0,10,9
2024-01-18,1014.0,63.5,37,39


In [71]:
# Output the DataFrame to a CSV file
merged_df.to_csv('output.csv')